I lernt from the following websites to finish the implementations.
Reference:
1. Hung-yi Lee. DRL Lecture 3: Q-learning (Basic Idea)
https://youtu.be/o_g9JUMw1Oc
2. Hung-yi Lee. DRL Lecture 4: Q-learning (Advanced Tips)
https://www.youtube.com/watch?v=2-zGCx4iv_k&t=980s
3. Tensorflow
https://www.tensorflow.org/api_docs/python/tf/keras/Model
4. Tom. Let's build a DQN: simple implementation
https://tomroth.com.au/dqn-simple/
5. Reinforcement Learning tutorial
https://pylessons.com/CartPole-DDDQN/

In [1]:
import gym
import slimevolleygym
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline
import tensorflow as tf
from random import sample

In [2]:
# Setting up the environment 
env = gym.make("SlimeVolley-v0")

In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape= (12,)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(32, activation='relu'),  
  tf.keras.layers.Dense(8)
])
model.compile(optimizer='adam',
              loss="mse",
              metrics=['accuracy'])



# Target network
model2 = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape= (12,)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(32, activation='relu'),  
  tf.keras.layers.Dense(8)
])
model2.compile(optimizer='adam',
              loss="mse",
              metrics=['accuracy'])

In [4]:
def action_inverse(num):
    if num == 0:
        return np.array([0,0,0])
    elif num == 1:
        return np.array([1,0,0])
    elif num == 2:
        return np.array([0,1,0])
    elif num == 3:
        return np.array([0,0,1])
    elif num == 4:
        return np.array([1,1,0])
    elif num == 5:
        return np.array([1,0,1])
    elif num == 6:
        return np.array([0,1,1])
    elif num == 7:
        return np.array([1,1,1])

In [5]:
def replay(buffer, minibatch_size,timestep,epsoide_num):
    if timestep <= 5000:
        return model
    else:
        minibatch = np.random.choice(buffer, minibatch_size, replace=False)
        s_l =      np.array(list(map(lambda x: x['s'], minibatch)))
        a_l =      np.array(list(map(lambda x: x['a'], minibatch)))
        r_l =      np.array(list(map(lambda x: x['r'], minibatch)))
        sprime_l = np.array(list(map(lambda x: x['sprime'], minibatch)))
        done_l   = np.array(list(map(lambda x: x['done'], minibatch)))
        current_qs = model.predict(s_l)


        if epsoide_num % 10 == 0 and epsoide_num != 0:
            model2.set_weights(model.get_weights())   
        qvals_sprime_l = model2.predict(sprime_l) # includes the other actions, states
        # q-update

        for i,(s,a,r,qvals_sprime, done) in enumerate(zip(s_l,a_l,r_l,qvals_sprime_l, done_l)): 
            if not done:  target = r + np.max(qvals_sprime)
            else:         target = r
            # Update Q value for given state
            current_qs[i][a] = target
            # And append to our training data
        model.fit(s_l,current_qs, epochs=1,batch_size= minibatch_size,verbose=0)
        return model

In [6]:
from random import randrange

# hyper parameter
n_episodes = 3500
epsilon = 0.9
minibatch_size = 40
r_sums = []  # stores rewards of each epsiode 
buffer = [] # replay memory holds s, a, r, s'
win_weight_eps_tstep = []
mem_max_size = 30000
init_seed=123
timestep = 0
eps = 0
policy0 = slimevolleygym.BaselinePolicy()
np.random.seed(123)


for n in range(n_episodes): 
    env.seed(seed=init_seed+n)
    s = env.reset()
    s_opponent = s
    done=False
    r_sum = 0
    while not done: 
        # Uncomment this to see the agent learning
        # env.render()
        # Feedforward pass for current state to get predicted q-values for all actions 
        qvals_s = model(s.reshape(1,12)).numpy()
        # Choose action to be epsilon-greedy
        if np.random.random() < epsilon:  
            a = randrange(8)
        else:                             
            a = np.argmax(qvals_s)
        action1 = policy0.predict(s_opponent) #opponent policy
        a_transfor = action_inverse(a)  
        # Take step, store results 
        sprime, r, done, info = env.step(a_transfor, action1)
        r_sum += r 
        # add to memory, respecting memory buffer limit 
        if len(buffer) > mem_max_size:
            buffer.pop(0)
        buffer.append({"s":s,"a":a,"r":r,"sprime":sprime,"done":done})
        # Update state
        s=sprime
        s_opponent = info['otherObs']
        # Train the nnet that approximates q(s,a), using the replay memory
        model=replay(buffer, minibatch_size = minibatch_size, timestep = timestep, epsoide_num = n)
        # Decrease epsilon until we hit a target threshold 
        timestep += 1
        if timestep % 5000 == 0 and timestep != 0:
            print("timestep",timestep)
    if r_sum > 0:
        win_weight_eps_tstep.append((model.get_weights(),n,timestep))    
    epsilon -= 0.0005
    if epsilon <= 0.0005:
        eps += 0.001
        epsilon = 0.001
        if eps == 0.001:
            print("In the timestep of ", timestep, ", epsilon is fixed at",epsilon)     
    #print("Total reward:", r_sum)
    r_sums.append(r_sum)
    if n % 100 == 0: print(n)
    if n % 10 == 0: 
        print("r_score", np.round(np.mean(np.array(r_sums)), 3), "±", 2*np.round(np.std(np.array(r_sums)), 3), "vs", "Baseline", "over", n+1, "trials.")
        print("rsums_lastest10",r_sums[-10:])

0
r_score -5.0 ± 0.0 vs Baseline over 1 trials.
rsums_lastest10 [-5]
timestep 5000
r_score -4.545 ± 1.782 vs Baseline over 11 trials.
rsums_lastest10 [-5, -5, -2, -4, -5, -5, -5, -5, -4, -5]
timestep 10000
r_score -4.714 ± 1.4 vs Baseline over 21 trials.
rsums_lastest10 [-5, -5, -5, -5, -4, -5, -5, -5, -5, -5]
timestep 15000
r_score -4.806 ± 1.182 vs Baseline over 31 trials.
rsums_lastest10 [-5, -5, -5, -5, -5, -5, -5, -5, -5, -5]
timestep 20000
r_score -4.854 ± 1.042 vs Baseline over 41 trials.
rsums_lastest10 [-5, -5, -5, -5, -5, -5, -5, -5, -5, -5]
timestep 25000
timestep 30000
r_score -4.882 ± 0.942 vs Baseline over 51 trials.
rsums_lastest10 [-5, -5, -5, -5, -5, -5, -5, -5, -5, -5]
timestep 35000
r_score -4.869 ± 0.922 vs Baseline over 61 trials.
rsums_lastest10 [-5, -5, -5, -5, -4, -5, -5, -5, -4, -5]
timestep 40000
r_score -4.859 ± 0.966 vs Baseline over 71 trials.
rsums_lastest10 [-5, -5, -5, -5, -5, -5, -5, -5, -3, -5]
timestep 45000
r_score -4.864 ± 0.93 vs Baseline over 81 t

timestep 375000
r_score -4.84 ± 0.82 vs Baseline over 651 trials.
rsums_lastest10 [-5, -5, -5, -3, -5, -5, -5, -5, -4, -5]
timestep 380000
r_score -4.843 ± 0.814 vs Baseline over 661 trials.
rsums_lastest10 [-5, -5, -5, -5, -5, -5, -5, -5, -5, -5]
timestep 385000
r_score -4.845 ± 0.81 vs Baseline over 671 trials.
rsums_lastest10 [-5, -5, -5, -5, -5, -5, -5, -5, -5, -5]
timestep 390000
r_score -4.846 ± 0.806 vs Baseline over 681 trials.
rsums_lastest10 [-5, -5, -5, -5, -4, -5, -5, -5, -5, -5]
timestep 395000
r_score -4.848 ± 0.802 vs Baseline over 691 trials.
rsums_lastest10 [-5, -5, -5, -5, -5, -5, -5, -5, -5, -5]
timestep 400000
700
r_score -4.845 ± 0.806 vs Baseline over 701 trials.
rsums_lastest10 [-5, -5, -5, -4, -4, -5, -5, -4, -4, -5]
timestep 405000
r_score -4.841 ± 0.812 vs Baseline over 711 trials.
rsums_lastest10 [-5, -5, -5, -4, -5, -4, -5, -5, -4, -4]
timestep 410000
timestep 415000
r_score -4.84 ± 0.812 vs Baseline over 721 trials.
rsums_lastest10 [-5, -4, -5, -5, -5, -4, 

r_score -4.861 ± 0.766 vs Baseline over 1291 trials.
rsums_lastest10 [-5, -4, -5, -5, -5, -3, -5, -4, -5, -4]
timestep 745000
1300
r_score -4.862 ± 0.762 vs Baseline over 1301 trials.
rsums_lastest10 [-5, -5, -5, -5, -5, -5, -5, -5, -5, -5]
timestep 750000
r_score -4.863 ± 0.76 vs Baseline over 1311 trials.
rsums_lastest10 [-5, -5, -5, -5, -5, -5, -5, -5, -5, -5]
timestep 755000
r_score -4.862 ± 0.76 vs Baseline over 1321 trials.
rsums_lastest10 [-5, -4, -5, -5, -5, -5, -5, -4, -5, -5]
timestep 760000
r_score -4.863 ± 0.76 vs Baseline over 1331 trials.
rsums_lastest10 [-5, -5, -5, -5, -5, -5, -5, -5, -5, -4]
timestep 765000
r_score -4.861 ± 0.766 vs Baseline over 1341 trials.
rsums_lastest10 [-5, -3, -5, -5, -5, -5, -5, -5, -4, -5]
timestep 770000
r_score -4.862 ± 0.762 vs Baseline over 1351 trials.
rsums_lastest10 [-5, -5, -5, -5, -5, -5, -5, -5, -5, -5]
timestep 775000
r_score -4.863 ± 0.762 vs Baseline over 1361 trials.
rsums_lastest10 [-5, -5, -5, -5, -5, -5, -5, -5, -5, -4]
timest

timestep 1100000
r_score -4.867 ± 0.74 vs Baseline over 1931 trials.
rsums_lastest10 [-5, -5, -5, -5, -5, -4, -5, -5, -5, -5]
timestep 1105000
r_score -4.867 ± 0.74 vs Baseline over 1941 trials.
rsums_lastest10 [-5, -5, -4, -5, -4, -5, -5, -5, -5, -5]
timestep 1110000
r_score -4.866 ± 0.742 vs Baseline over 1951 trials.
rsums_lastest10 [-5, -4, -5, -5, -4, -5, -5, -5, -5, -5]
timestep 1115000
r_score -4.866 ± 0.74 vs Baseline over 1961 trials.
rsums_lastest10 [-5, -4, -5, -5, -5, -5, -5, -5, -5, -5]
timestep 1120000
r_score -4.867 ± 0.74 vs Baseline over 1971 trials.
rsums_lastest10 [-5, -5, -5, -5, -5, -5, -5, -5, -5, -5]
timestep 1125000
r_score -4.866 ± 0.744 vs Baseline over 1981 trials.
rsums_lastest10 [-5, -5, -5, -5, -4, -5, -5, -5, -5, -3]
timestep 1130000
r_score -4.866 ± 0.744 vs Baseline over 1991 trials.
rsums_lastest10 [-5, -5, -5, -5, -5, -5, -4, -5, -4, -5]
timestep 1135000
timestep 1140000
2000
r_score -4.867 ± 0.742 vs Baseline over 2001 trials.
rsums_lastest10 [-5, -5

timestep 1460000
r_score -4.868 ± 0.736 vs Baseline over 2571 trials.
rsums_lastest10 [-5, -5, -5, -5, -5, -5, -5, -5, -5, -5]
timestep 1465000
r_score -4.868 ± 0.736 vs Baseline over 2581 trials.
rsums_lastest10 [-5, -5, -5, -5, -5, -5, -4, -5, -5, -5]
timestep 1470000
r_score -4.869 ± 0.734 vs Baseline over 2591 trials.
rsums_lastest10 [-5, -5, -5, -5, -5, -5, -5, -5, -5, -5]
timestep 1475000
timestep 1480000
2600
r_score -4.869 ± 0.734 vs Baseline over 2601 trials.
rsums_lastest10 [-5, -5, -5, -5, -5, -5, -5, -5, -5, -5]
timestep 1485000
r_score -4.87 ± 0.732 vs Baseline over 2611 trials.
rsums_lastest10 [-5, -5, -5, -5, -5, -5, -5, -5, -5, -5]
timestep 1490000
r_score -4.87 ± 0.732 vs Baseline over 2621 trials.
rsums_lastest10 [-5, -5, -5, -4, -5, -5, -5, -5, -5, -5]
timestep 1495000
r_score -4.87 ± 0.73 vs Baseline over 2631 trials.
rsums_lastest10 [-5, -5, -5, -5, -5, -5, -5, -5, -5, -5]
timestep 1500000
r_score -4.871 ± 0.73 vs Baseline over 2641 trials.
rsums_lastest10 [-5, -5,

timestep 1825000
timestep 1830000
r_score -4.868 ± 0.74 vs Baseline over 3211 trials.
rsums_lastest10 [-5, -5, -5, -4, -5, -5, -5, -5, -5, -5]
timestep 1835000
r_score -4.868 ± 0.74 vs Baseline over 3221 trials.
rsums_lastest10 [-5, -5, -5, -4, -5, -5, -5, -5, -5, -5]
timestep 1840000
r_score -4.868 ± 0.74 vs Baseline over 3231 trials.
rsums_lastest10 [-5, -5, -5, -5, -5, -5, -5, -5, -5, -5]
timestep 1845000
r_score -4.868 ± 0.74 vs Baseline over 3241 trials.
rsums_lastest10 [-4, -5, -5, -5, -5, -5, -5, -5, -5, -5]
timestep 1850000
r_score -4.868 ± 0.738 vs Baseline over 3251 trials.
rsums_lastest10 [-5, -4, -5, -5, -5, -5, -5, -5, -5, -5]
timestep 1855000
r_score -4.869 ± 0.738 vs Baseline over 3261 trials.
rsums_lastest10 [-5, -5, -5, -5, -5, -5, -5, -5, -5, -5]
timestep 1860000
r_score -4.869 ± 0.738 vs Baseline over 3271 trials.
rsums_lastest10 [-5, -5, -5, -4, -5, -5, -5, -5, -5, -5]
timestep 1865000
timestep 1870000
r_score -4.869 ± 0.738 vs Baseline over 3281 trials.
rsums_laste

In [7]:
print("r_score", np.round(np.mean(np.array(r_sums)), 3), "±", 2*np.round(np.std(np.array(r_sums)), 3), "vs", "Baseline", "over", n+1, "trials.")
print("rsums_lastest10",r_sums[-10:])

r_score -4.871 ± 0.728 vs Baseline over 3500 trials.
rsums_lastest10 [-5, -5, -4, -4, -5, -5, -5, -5, -4, -5]


In [14]:
import pickle
# Store data (serialize)
with open('NEW_NDQN_batch_size.pickle', 'wb') as handle:
    pickle.dump(r_sums +[timestep,], handle)

In [15]:
with open('NEW_NDQN_batch_size.pickle', 'rb') as handle:
    b = pickle.load(handle)

In [16]:
print(len(b))
b[-1]

3501


1995217